# Analysis code for the paper "B1+ shimming in the cervical spinal cord at 7T"

## Data

The data can be downloaded at https://openneuro.org/datasets/ds004906

The structure of the input dataset is as follows (JSON sidecars are not listed for clarity):
~~~
ds004906
├── CHANGES
├── README
├── dataset_description.json
├── participants.json
├── participants.tsv
├── sub-01
│   ├── anat
│   │   ├── sub-01_acq-CP_T1w.nii.gz
│   │   ├── sub-01_acq-CP_T2starw.nii.gz
│   │   ├── sub-01_acq-CoV_T1w.nii.gz
│   │   ├── sub-01_acq-CoV_T2starw.nii.gz
│   │   ├── sub-01_acq-SAReff_T2starw.nii.gz
│   │   ├── sub-01_acq-patient_T2starw.nii.gz
│   │   ├── sub-01_acq-phase_T2starw.nii.gz
│   │   ├── sub-01_acq-target_T2starw.nii.gz
│   │   ├── sub-01_acq-volume_T2starw.nii.gz
│   └── fmap
│       ├── sub-01_acq-anatCP_TB1TFL.nii.gz
│       ├── sub-01_acq-anatCoV_TB1TFL.nii.gz
│       ├── sub-01_acq-anatSAReff_TB1TFL.nii.gz
│       ├── sub-01_acq-anatpatient_TB1TFL.nii.gz
│       ├── sub-01_acq-anatphase_TB1TFL.nii.gz
│       ├── sub-01_acq-anattarget_TB1TFL.nii.gz
│       ├── sub-01_acq-anatvolume_TB1TFL.nii.gz
│       ├── sub-01_acq-fampCP_TB1TFL.nii.gz
│       ├── sub-01_acq-fampCoV_TB1TFL.nii.gz
│       ├── sub-01_acq-fampSAReff_TB1TFL.nii.gz
│       ├── sub-01_acq-famppatient_TB1TFL.nii.gz
│       ├── sub-01_acq-fampphase_TB1TFL.nii.gz
│       ├── sub-01_acq-famptarget_TB1TFL.nii.gz
│       └── sub-01_acq-fampvolume_TB1TFL.nii.gz
├── sub-02
├── sub-03
├── sub-04
└── sub-05
~~~


## Overview of processing pipeline

For each subject:

- Segment the spinal cord on GRE scan
- Label vertebral levels on GRE scan using existing manual disc labels
- Extract the signal intensity on the GRE scan within the spinal cord
- Register each B1 map (CP, CoV, etc.) to the GRE scan
- Apply the computed warping field to bring the segmentation and vertebral levels to the B1 map
- Convert the B1 map to nT/V units
- Extract the B1 map value within the spinal cord

```{note}
Here is a note!
```
Slow processes are indicated with the emoji ⏳

In [ ]:
# Necessary imports

import os
import re
import json
import subprocess
import glob
import numpy as np
# import matplotlib.pyplot as plt
# from PIL import Image
# from IPython.display import display
# from tabulate import tabulate
import nibabel as nib
# import pandas as pd

In [ ]:
# Download data and define path variables

!datalad install https://github.com/OpenNeuroDatasets/ds004906.git
os.chdir("ds004906")
# !datalad get . # uncomment for production
!datalad get sub-01/  # debugging
# Get derivatives containing manual labels
!datalad get derivatives

In [ ]:
# Define useful variables

path_data = os.getcwd()
print(f"path_data: {path_data}")
path_qc = os.path.join(path_data, "qc")
# shim_modes = ["CP", "CoV", "patient", "phase", "SAReff", "target", "volume"]
shim_modes = ["CP", "CoV"]  # debugging
print(f"shim_modes: {shim_modes}")
subjects = sorted(glob.glob("sub-*"))
print(f"subjects: {subjects}")

In [ ]:
# Run segmentation on GRE scan

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    !sct_deepseg_sc -i {subject}_acq-CoV_T2starw.nii.gz -c t2 -qc {path_qc}

## Verify QC report (GRE segmentation)

Open the quality control (QC) report located under `ds004906/qc/index.html`. Make sure the spinal cord segmentations are correct before resuming the analysis.

In [ ]:
# Crop GRE scan for faster processing and better registration results

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    !sct_crop_image -i {subject}_acq-CoV_T2starw.nii.gz -m {subject}_acq-CoV_T2starw_seg.nii.gz -dilate 20x20x0 -o {subject}_acq-CoV_T2starw_crop.nii.gz
    !sct_crop_image -i {subject}_acq-CoV_T2starw_seg.nii.gz -m {subject}_acq-CoV_T2starw_seg.nii.gz -dilate 20x20x0 -o {subject}_acq-CoV_T2starw_crop_seg.nii.gz

In [ ]:
# Label vertebrae on GRE scan

# Given the low resolution of the GRE scan, the automatic detection of C2-C3 disc is unreliable. Therefore we need to use the manual disc labels that are part of the dataset.
for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    fname_label_discs = os.path.join(path_data, "derivatives", "labels", subject, "anat", f"{subject}_acq-CoV_T2starw_label-discs_dseg.nii.gz")
    !sct_label_utils -i {subject}_acq-CoV_T2starw_crop_seg.nii.gz -disc {fname_label_discs} -o {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz
    # Generate QC report to assess labeled segmentation
    !sct_qc -i {subject}_acq-CoV_T2starw_crop.nii.gz -s {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -p sct_label_vertebrae -qc {path_qc} -qc-subject {subject}

In [ ]:
# Register the other shim methods to the GRE CoV scan

# TODO

In [ ]:
# Extract the signal intensity on the GRE scan within the spinal cord

for subject in subjects:
    # TODO: loop across other shim methods    
    os.chdir(os.path.join(path_data, subject, "anat"))
    !sct_extract_metric -i {subject}_acq-CoV_T2starw_crop.nii.gz -f {subject}_acq-CoV_T2starw_crop_seg.nii.gz -method wa -vert 1:9 -vertfile {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -append 1 -perslice 1 -o gre_CoV.csv


In [ ]:
# Register TFL B1maps to the GRE scan ⏳

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        !sct_register_multimodal -i {subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -d ../anat/{subject}_acq-CoV_T2starw_crop.nii.gz -dseg ../anat/{subject}_acq-CoV_T2starw_crop_seg.nii.gz -param step=1,type=im,algo=slicereg,metric=CC -qc {path_qc}

## Verify QC report (B1maps to GRE registration)

Open the QC report located under `ds004906/qc/index.html`. Make sure the registration are correct before resuming the analysis.

In [ ]:
# Warping spinal cord segmentation and vertebral level to each B1 map

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        !sct_apply_transfo -i ../anat/{subject}_acq-CoV_T2starw_crop_seg.nii.gz -d {subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -w warp_{subject}_acq-CoV_T2starw_crop2{subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -x linear -o {subject}_acq-anat{shim_mode}_TB1TFL_seg.nii.gz
        !sct_apply_transfo -i ../anat/{subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -d {subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -w warp_{subject}_acq-CoV_T2starw_crop2{subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -x nn -o {subject}_acq-anat{shim_mode}_TB1TFL_seg_labeled.nii.gz

In [ ]:
# Convert the B1 map to nT/V units (by Kyle Gilbert)

# GAMMA = 2.675e8
# B1eff_mag = (AcquiredFA ./ RequestedFA) .* (pi ./ (GAMMA .* 1e-3 .* VoltageAtSocket)); % [T/V]
# B1eff_mag = B1eff_mag .* 1e9; % [T/V] to [nT/V]
# The constants sum up to 130.492, so to convert the B1map to nT/V, it has to be divided by 10 (to get it back into units of FA)
# then multiplied by 130.492 and divided by the VoltageAtSocket

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        # Fetch the reference voltage from the JSON sidecar to the TFL B1map sequence
        with open(f"{subject}_acq-famp{shim_mode}_TB1TFL.json", "r") as f:
            metadata = json.load(f)
            ref_voltage = metadata.get("TxRefAmp", "N/A")
            print(f"ref_voltage: {ref_voltage}")

        voltage_at_socket = ref_voltage * 10**-0.095  # TODO: justify this 0.095 number https://github.com/shimming-toolbox/rf-shimming-7t/issues/22
        # VoltageAtSocket=np.around(VoltageAtSocket, decimals=2)

        # Divide the flip angle map by a factor 10 to get it in the proper unit (https://github.com/shimming-toolbox/rf-shimming-7t/issues/22)
        !sct_maths -i {subject}_acq-famp{shim_mode}_TB1TFL.nii.gz -div 10 -o {subject}_acq-famp{shim_mode}_TB1TFL_nTpV.nii.gz

        # Divide the flip angle map by the voltage at the socket
        !sct_maths -i {subject}_acq-famp{shim_mode}_TB1TFL_nTpV.nii.gz -div {voltage_at_socket} -o {subject}_acq-famp{shim_mode}_TB1TFL_nTpV.nii.gz

        # Multiply the output by 130.492, which corresponds to ...? (https://github.com/shimming-toolbox/rf-shimming-7t/issues/22)
        !sct_maths -i {subject}_acq-famp{shim_mode}_TB1TFL_nTpV.nii.gz -mul 130.492 -o {subject}_acq-famp{shim_mode}_TB1TFL_nTpV.nii.gz

In [ ]:
# Extract B1+ value along the spinal cord

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        !sct_extract_metric -i {subject}_acq-famp{shim_mode}_TB1TFL_nTpV.nii.gz -f {subject}_acq-anat{shim_mode}_TB1TFL_seg.nii.gz -method wa -vert 1:9 -vertfile {subject}_acq-anat{shim_mode}_TB1TFL_seg_labeled.nii.gz -perslice 1 -o TB1TFL_{shim_mode}.csv

In [ ]:
# Make figure of B1+ values along the spinal cord across shim methods
# WIP

import pandas as pd
import matplotlib.pyplot as plt
import glob

subject='sub-01'
os.chdir(os.path.join(path_data, subject, "fmap"))

# Pattern to match all files starting with 'TB1TFL_' and ending with '.csv'
file_pattern = f"TB1TFL_*.csv"


# Initialize a matplotlib figure
plt.figure(figsize=(10, 6))

# Iterate over all files that match the pattern
for file_path in glob.glob(file_pattern):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Assuming 'WA()' is the column name, extract data from this column
    wa_data = df['WA()']
    
    # Plot the data
    plt.plot(wa_data, label=file_path.split('/')[-1])  # Using file name as label

# Adding title, labels, and legend
plt.title('Data from Column WA() Across Multiple Files')
plt.xlabel('Index')
plt.ylabel('Values')
plt.legend()
plt.grid(True)

# Display the plot
plt.show()